# HMC-GRAD: HANDWRITTEN MULTIPLE-CHOICE TEST GRADER
The implementation of HMC-Grad, employing OpenCV for image preprocessing and PyTorch for training a convolutional neural network on the EMNIST dataset for image classification.

## IMPORT MODULES AND SET GLOBALS

In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.2 M

In [2]:
# For downloading model weights
import os

# For image preprocessing
import cv2
import numpy as np
from sklearn.cluster import KMeans

# For the image classifier
import torch
import torch.nn as nn
import torch.nn.functional as F

# For the interface
import gradio as gr

# For formatting data
import pandas as pd

In [3]:
# Globals Constants:

# For preprocessing
IMAGE_WIDTH = 1125
IMAGE_HEIGHT = 1500
THRESH_BLOCK_SIZE = 11
THRESH_CONSTANT = 5
LINE_LENGTH = 5000

# For ROI extraction
MAX_COMPONENT_MERGE_DISTANCE = 30
MIN_COMPONENT_SIDE = 15
Y_EPSILON = 25
NUMBER_OF_COLUMNS = 2

# For image classification
PREDICTION_TO_STRING = ["A", "B", "C", "D"]

## THE CLASSIFIER
Prepare the Classifier Model.

In [21]:
# Download the model's state dictionary from repository
GITHUB_URL = "https://raw.githubusercontent.com/GabrielEdradan/HMC-Grad/main/image_classifier.pth"
MODEL_PATH = "/content/model.pth"
os.system(f"wget {GITHUB_URL} -O {MODEL_PATH}")
model_state_dict = torch.load(MODEL_PATH)

In [22]:
# Define the model class
class ABCDClassifier(nn.Module):

  def __init__(self):
    super(ABCDClassifier, self).__init__()

    self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(32 * 4 * 4, 128)
    self.fc2 = nn.Linear(128, 4)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 32 * 4 * 4)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)

    return F.softmax(x, dim=1)

In [23]:
# Load the model
model = ABCDClassifier()
model.load_state_dict(model_state_dict)

<All keys matched successfully>

## THE IMAGE PROCESSOR

### The Image Processor Functions

In [24]:
# The main function used for the interface.
# Takes in an array of strings (image paths), and an array of chars (correction key)
# Returns four str file paths to csv files: correctness, scores, item analysis, and score analysis
def process_image_set(image_paths_array, correction_key):
  correctness = [] # an array of binary int arrays, of length len(image_paths_array)
  scores = [] # an array of ints, of length len(image_paths_array)
  item_analysis = [0] * len(correction_key) # an array of ints
  score_analysis = [0] * (len(correction_key) + 1) # an array of ints

  for i in range(len(image_paths_array)):
    process_image(image_paths_array[i], correctness, scores, item_analysis, score_analysis, correction_key)

  # Formatting data
  # Define csv file paths
  student_correctness_csv_path = "student_correctness.csv"
  student_scores_csv_path = "student_scores.csv"
  item_analysis_csv_path = "item_analysis.csv"
  score_analysis_csv_path = "score_analysis.csv"
  merged_xlsx_path = "merged_data.xlsx"

  # For correctness
  transposed_data = list(map(list, zip(*correctness))) # Transpose the data to have students as columns
  columns = [f"Student {i+1}" for i in range(len(correctness))] # Define the columns
  correctness_df = pd.DataFrame(transposed_data, columns=columns) # Create the DataFrame
  correctness_df.insert(0, "Item Number", range(1, len(correctness[0]) + 1)) # Add the item number column
  correctness_df.to_csv(student_correctness_csv_path, index=False) # Save

  # For student scores
  columns = ["Score"] # Define the columns
  scores_df = pd.DataFrame(scores, columns=columns) # Create the DataFrame
  scores_df.insert(0, "Student Number", range(1, len(scores) + 1)) # Add the student number column
  scores_df.to_csv(student_scores_csv_path, index=False) # Save

  # For item analysis
  columns = ["Number of Correct Answers"] # Define the columns
  item_analysis_df = pd.DataFrame(item_analysis, columns=columns) # Create the DataFrame
  item_analysis_df.insert(0, "Item Number", range(1, len(item_analysis) + 1)) # Add the student number column
  item_analysis_df.to_csv(item_analysis_csv_path, index=False) # Save

  # For score analysis
  columns = ["Number of Students"] # Define the columns
  score_analysis_df = pd.DataFrame(score_analysis, columns=columns) # Create the DataFrame
  score_analysis_df.insert(0, "Score", range(0, len(score_analysis))) # Add the student number column
  score_analysis_df.to_csv(score_analysis_csv_path, index=False) # Save

  # For merging CSV into into XLSX
  # Create a writer to save multiple dataframes to a single XLSX file
  with pd.ExcelWriter(merged_xlsx_path) as writer:
      # Write each dataframe to a different sheet
      correctness_df.to_excel(writer, sheet_name="Correctness", index=False)
      scores_df.to_excel(writer, sheet_name="Scores", index=False)
      item_analysis_df.to_excel(writer, sheet_name="Item Analysis", index=False)
      score_analysis_df.to_excel(writer, sheet_name="Score Analysis", index=False)

  return student_correctness_csv_path, student_scores_csv_path, item_analysis_csv_path, score_analysis_csv_path, merged_xlsx_path


# A helper function for readability
# Takes in an image path, the four arrays to be modified and an array of chars
# Void return value, modifies the four arrays directly
def process_image(img_path, img_cor_arr, img_scr_arr, itm_ana_arr, scr_ana_arr, correction_key):
  base_image = cv2.imread(img_path)

  # Preprocessing
  segmentation_image, ocr_image, processing_error = preprocess(base_image)
  if processing_error: # Check for exception
    invalid_num_of_items(img_scr_arr, "PROCESSING ERROR")
    return

  # ROI Extraction
  rois, extraction_error = extract_rois(segmentation_image, ocr_image, len(correction_key))
  if extraction_error: # Check for exception
    invalid_num_of_items(img_scr_arr, "EXTRACTION ERROR")
    return

  # Classification
  item_answers = classify_rois(rois)

  if len(item_answers) != len(correction_key): # Check for exception (extra layer of safety)
    invalid_num_of_items(img_scr_arr, "NUM OF ITEM ANSWERS != CORRECTION KEY")
    return

  # Grading and Analysis
  grade_and_analyze(item_answers, img_cor_arr, img_scr_arr, itm_ana_arr, scr_ana_arr, correction_key)


# Function for error logic: sets the score to -1 (-1 score means invalid image)
# Takes in the array to be modified (score array)
# Void return value, modifies the array directly
def invalid_num_of_items(score_array, error_string):
  score_array.append(-1)
  print(f"ERROR: {error_string}")

### Image Preprocessing Functions

In [25]:
# Takes in an image (the base image)
# Returns an image for segmentation, an image for OCR, and a boolean for error handling
def preprocess(input_image):
  # ------------------------------------ FOR OCR IMAGE------------------------------------ #
  resized_image = cv2.resize(input_image, (IMAGE_WIDTH, IMAGE_HEIGHT)) # MOVED FROM INTERFACE TO HERE
  gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
  threshed_gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, THRESH_BLOCK_SIZE, THRESH_CONSTANT)
  opened = cv2.bitwise_not(cv2.morphologyEx(cv2.bitwise_not(threshed_gray), cv2.MORPH_OPEN, np.ones((2,2),np.uint8), iterations=1))
  ocr_image = remove_lines(opened)

  # ----------------------------------- FOR HEADER MASK ----------------------------------- #
  blur = cv2.GaussianBlur(gray, (9,9), 0)
  threshed_blur = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, THRESH_BLOCK_SIZE, THRESH_CONSTANT)
  removed_lines = remove_lines(threshed_blur, 6)

  # Determine divider y positions
  intensely_dilated = cv2.dilate(cv2.bitwise_not(removed_lines), cv2.getStructuringElement(cv2.MORPH_RECT, (2000, 40)), iterations=1)
  black_ys = np.where(np.any(intensely_dilated == 0, axis=1))[0].tolist()
  to_be_removed = []
  for i in range(1, len(black_ys)):
    if black_ys[i] - black_ys[i - 1] == 1:
      to_be_removed.append(i)
  stripe_positions = [black_ys[idx] for idx in range(len(black_ys)) if not idx in to_be_removed]

  # Determine which stripe is the header border
  num_of_stripe_positions = len(stripe_positions)
  header_border = 0
  match num_of_stripe_positions:
    case 1:
      header_border = stripe_positions[0]
    case 2:
      header_border = stripe_positions[0] if stripe_positions[1] > 500 else stripe_positions[1]
    case _:
      found = False
      for stripe_position in stripe_positions:
        if stripe_position > 100 and stripe_position < 500:
          header_border = stripe_position
          found = True
          continue
      if not found: # If there is no stripe, consider the image invalid (subject to change)
        return [0], [0], True

  # Create a mask based on header border
  mask = np.ones(intensely_dilated.shape, dtype=np.uint8) * 255
  mask[header_border:, :] = 0

  # ---------------------------------- FOR COMPONENT FINDING IMAGE ---------------------------------- #
  masked_removed_lines = cv2.bitwise_or(removed_lines, mask)
  segmentation_image = cv2.dilate(cv2.bitwise_not(masked_removed_lines), cv2.getStructuringElement(cv2.MORPH_RECT, (10, 6)), iterations=1)

  return segmentation_image, ocr_image, False

In [26]:
# A helper function for the preprocess function
# Takes in an image, and optionally line thickness (an int that determines how erased the notepad lines are)
# Returns a modified image, where the notepad lines are attempted to be erased
def remove_lines(input_image, thickness=2):
  edges = cv2.Canny(input_image, 200, 200)
  lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

  # Check for when lines are not found
  if len(lines) == 0:
    return input_image

  # Creates a mask based on the lines found by HoughLines
  line_mask = np.zeros_like(input_image)
  for line in lines:
    rho, theta = line[0]
    if theta < 1.0 or theta > 2.0:
      continue
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + LINE_LENGTH * (-b))
    y1 = int(y0 + LINE_LENGTH * (a))
    x2 = int(x0 - LINE_LENGTH * (-b))
    y2 = int(y0 - LINE_LENGTH * (a))
    cv2.line(line_mask, (x1, y1), (x2, y2), (255, 255, 255), 2)

  # Dilates the lines vertically based on thickness parameter
  dilation_kernel = np.ones((thickness, 1),np.uint8)
  line_mask = cv2.dilate(line_mask, dilation_kernel, iterations=1)

  # Subtracts the mask from the base image and applies MORPH_OPEN to denoise
  sub_result = cv2.bitwise_or(input_image, line_mask)
  final_result = cv2.bitwise_not(cv2.morphologyEx(cv2.bitwise_not(sub_result), cv2.MORPH_OPEN, np.ones((2, 2),np.uint8), iterations=1))
  return final_result

### Region of Interest Extraction Functions

In [27]:
# Takes in an image for segmentation, an image for OCR, and the target number of ROIs
# Returns an array of RIOs (of shape [y, x]), and a boolean for error handling
def extract_rois(segmentation_image, ocr_image, num_of_items):
  # Get the components in the input image
  _, _, stats, centroids = cv2.connectedComponentsWithStats(segmentation_image, connectivity=4)

  # Define all bounds (stats excluding area)
  all_bounds = [stat[:4].tolist() for stat in stats]

  # Remove the background from bounds and centroids
  all_bounds.pop(0)
  centroids = centroids.tolist()
  centroids.pop(0)

  # Find components that are close to each other and merge them
  nearby_component_pairs = find_nearby_pairs(centroids, MAX_COMPONENT_MERGE_DISTANCE)
  mergeable_components = find_mergeable_components(nearby_component_pairs)
  merged_bounds = merge_groups(mergeable_components, all_bounds)

  # Make an array of bounds, exclude bounds that were used in merging and bounds that are too small
  component_bounds = [bound[:4] for index, bound in enumerate(all_bounds) if not any(index in group for group in mergeable_components) and (bound[2] > MIN_COMPONENT_SIDE and bound[3] > MIN_COMPONENT_SIDE)]

  # Add the merged bounds
  component_bounds.extend(merged_bounds)

  # Sort components into two columns
  component_bounds = sort_into_columns(component_bounds)

  # At this point, components in each column typically have one or more components in the same y (y is within Y_EPSILON)
  # Remove components except the ones rightmost in each row
  component_bounds = filter_non_letters(component_bounds)

  # Convert bounds to ROIs
  rois = []
  for bound in component_bounds:
    x, y, w, h = bound
    roi_img = ocr_image[y:y+h, x:x+w]
    rois.append(roi_img)

  # Handle exception: If the number of ROIs found is not the same as the target, consider the image invalid
  if len(rois) != num_of_items:
    return [0], True

  return rois, False

In [28]:
# A helper function for the roi extraction function
# Takes in an array of centroids (component midpoints) and max merge distance
# Returns an array of tuples, representing nearby pairs of components
def find_nearby_pairs(centroids_array, max_merge_distance):
  nearby_pairs = []
  num_components = len(centroids_array)
  for i in range(num_components - 1):
    for j in range(i + 1, num_components):
      distance = np.linalg.norm(np.array(centroids_array[i]) - np.array(centroids_array[j]))
      if distance <= max_merge_distance:
        nearby_pairs.append((i, j))
  return nearby_pairs

In [29]:
# A helper function for the roi extraction function
# Takes in an array of nearby pairs
# Returns an array of sets, representing two or more components that are close to each other
def find_mergeable_components(nearby_pairs):
  groups = []
  for pair in nearby_pairs:
    group_found = False
    for group in groups:
      if any(component in group for component in pair): # Evaluates, for each pair component, if it is in the group
        group.update(pair)
        group_found = True
        break
    if not group_found:
      groups.append(set(pair))
  return groups

In [30]:
# A helper function for the roi extraction function
# Takes in an array of mergeable groups and all the component bounds
# Returns an array of the bounds that were the result of merging the mergeable groups
def merge_groups(groups, bounds):
  merged_bounds = []
  for group in groups:
    min_x = 5000
    min_y = 5000
    max_x = 0
    max_y = 0
    for component in group:
      x, y, w, h = bounds[component]
      min_x = min(min_x, x)
      min_y = min(min_y, y)
      max_x = max(max_x, x+w)
      max_y = max(max_y, y+h)
    merged_bounds.append([min_x, min_y, max_x - min_x, max_y - min_y])
  return merged_bounds

In [31]:
# A helper function for the roi extraction function
# Takes in an array of component bounds
# Returns an array of component bounds that are sorted into k columns
def sort_into_columns(component_bounds):
  # Determine the x coordinates of the bounds
  x_coordinates = [bound[0] for bound in component_bounds]
  x_coordinates = np.array(x_coordinates).reshape(-1, 1)

  # Set the optimal number of clusters (k) to NUMBER_OF_COLUMNS, as the number of columns is predefined
  optimal_k = NUMBER_OF_COLUMNS

  # Perform K-means clustering with k = NUMBER_OF_COLUMNS
  kmeans = KMeans(n_clusters=optimal_k, init="k-means++", max_iter=300, n_init=10, random_state=0)
  kmeans.fit(x_coordinates)

  # Group the components based on the cluster assignments
  grouped_components = [[] for _ in range(optimal_k)] # An array of two arrays
  for i, label in enumerate(kmeans.labels_): # kmeans.labels_ is an array of ints representing the label of each component
    grouped_components[label].append(component_bounds[i])

  # Sort into a single list
  sorted_components = []
  grouped_components = sorted(grouped_components, key=lambda group: group[0][0])
  for group in grouped_components:
    sorted_group = sorted(group, key=lambda component: component[1])
    sorted_components.extend(sorted_group)

  return sorted_components

In [32]:
# A helper function for the roi extraction function
# Takes in an array of component bounds
# Returns an array of component bounds that excludes positionally considered to be non-letters
def filter_non_letters(component_bounds):
  # Defines dictionaries mapping components to their origin-x and centroid-y
  comp_x_dict = {}
  for index, bound in enumerate(component_bounds):
    comp_x_dict[index] = bound[0]

  comp_cent_y_dict = {}
  for index, bound in enumerate(component_bounds):
    comp_cent_y_dict[index] = bound[1] + (bound[3] / 2)


  # Function that compares two keys and decides if the current key should be removed
  # Takes in the current key, the key to check against, a dictionary of component origin-x, and a dicitonary of component centroid-y
  # Modifies the array directly; returns True if the current key has been removed
  def check_key_for_removal(curr_key, key_to_check, x_dict, y_dict):
    if not key_to_check in y_dict or not curr_key in y_dict:
      return False
    if abs(y_dict[key_to_check] - y_dict[curr_key]) > Y_EPSILON:
      return False

    curr_key_x = x_dict[curr_key]
    key_to_check_x = x_dict[key_to_check]
    if key_to_check_x > curr_key_x:
      y_dict.pop(curr_key)
      return True

  # Based on the components centroid-y, determine which components are in the same "row," i.e. components whose centroid-y are within Y_EPSILON
  # If the components have significantly different centroid-y, ignore; else, check if the current one is to the left or to the right relatively
  # If the current component is to the left of some other component in the same "row," remove it from the comp_cent_y_dict
  dup_y_dict = comp_cent_y_dict.copy()
  for key in dup_y_dict:
    prev_prev_key = key - 2
    prev_key = key - 1
    next_key = key + 1
    if not key in comp_cent_y_dict:
      continue

    done = check_key_for_removal(key, prev_prev_key, comp_x_dict, comp_cent_y_dict)
    if done: continue # If the curr_key has been removed, go to the next key

    done = check_key_for_removal(key, prev_key, comp_x_dict, comp_cent_y_dict)
    if done: continue

    done = check_key_for_removal(key, next_key, comp_x_dict, comp_cent_y_dict)

  # Create an array of component bounds that only excludes what remains in the comp_cent_y_dict
  filtered_component_bounds = [bound for index, bound in enumerate(component_bounds) if index in comp_cent_y_dict]

  return filtered_component_bounds

### Image Classification Functions

In [33]:
# Takes in an array of ROIs (numpy arrays)
# Returns an array of item answers (letter string)
def classify_rois(rois):
  item_answers = []
  for roi in rois:
    item_answers.append(classify_roi(roi))
  return item_answers

In [34]:
# Takes in an ROI expressed as a numpy array of shape [y, x]
# Returns the classifiers classification of the ROI
def classify_roi(roi):
  # Preprocess ROI
  new_array = np.full((28, 28), 255, dtype=np.uint8)
  small_roi = cv2.resize(roi, (26, 26))
  new_array[1:27, 1:27] = small_roi

  roi = new_array
  roi = cv2.bitwise_not(roi) # Invert to fit model requirement
  roi = roi / 255.0 # Normalize

  roi = torch.from_numpy(roi) # Convert to tensor
  roi = roi.view(1, 1, 28, 28) # Reshape to fit model requirement
  roi = roi.to(torch.float32) # Change data type to fit model requirement

  # Classify the ROI
  model.eval()
  output = model(roi) # Output is a tensor with 4 floats representing class probabilities
  prediction = output.argmax(dim=1, keepdim=True).item() # Returns a number from 0 to 3, representing the most probable class
  predicted_letter = PREDICTION_TO_STRING[prediction] # Converts the number to the corresponding letter

  return predicted_letter

### Grading and Analysis Function

In [35]:
# Takes in 5 arrays and one string of text
# Has no return value. Instead, updates the arrays in place.
def grade_and_analyze(item_answers, img_cor_arr, img_scr_arr, itm_ana_arr, scr_ana_arr, correction_key):
  correction_array = []
  score = 0

  for i in range(len(correction_key)):
    if item_answers[i] == correction_key[i] or correction_key[i] == "X":
      correction_array.append(1) # Update correction array
      score += 1 # Update score
      itm_ana_arr[i] += 1 # Update item analysis
    else:
      correction_array.append(0) # Update correction array

  # Bring changes to the input arrays (except item analysis, which was updated during the loop)
  img_cor_arr.append(correction_array)
  img_scr_arr.append(score)
  scr_ana_arr[score] += 1 # Update score analysis

## THE INTERFACE
Use Gradio to create a user-friendly interface.

In [36]:
# Download the sample images for the demo interface
SAMPLE_IMAGES_FOLDER_URL = "https://raw.githubusercontent.com/GabrielEdradan/HMC-Grad/main/sample_images/"
IMAGE_SAVE_PATH = "/content/"

NUM_OF_SAMPLES = 2
for i in range(1, NUM_OF_SAMPLES + 1):
  suffix = f"sample_{i}.jpg"
  os.system(f"wget {SAMPLE_IMAGES_FOLDER_URL}{suffix} -O {IMAGE_SAVE_PATH}{suffix}")

In [37]:
# Define the interface function
# Takes in two inputs: the correction key (string) and the answer sheet images (Array[Array[image paths]])
def interface_function(correction_key_string, *images):

  # Set correction key
  correction_key = []
  for item in correction_key_string:
    if item in ["A", "B", "C", "D", "X"]:
      correction_key.append(item.upper())

  # Run the main function
  return process_image_set(images[0], correction_key)


# Define the Gradio interface

# Set the description string
desc ='''
This is the demo interface for the research project titled "HMC-Grad: Automating Handwritten Multiple-Choice Test Grading Using Computer Vision and Deep Learning" by Edradan, G., Serrano, D., and Tunguia, T.

Instructions:

First Input: Enter the correction key. It must be a continuous string of text, with each letter representing the correct answer for each item in consecutive order. The only letters accepted are A, B, C, and D, but an X can be written to represent an item that would accept any answer (bonus item).

Second Input: Upload the images of the papers to be evaluated and analyzed. The order of evaluation is based on the order in which the images are uploaded.

For better results, the following are recommended:

Regarding the documents to be evaluated:
  - Have substantial left and right margins
  - Provide a blank line between the header and the answers
  - Write the answers in capitals and in two columns
  - Avoid having the answers overlap with the notepad lines
  - Have significant space between the numbers and the letters
  - Write the item numbers smaller than the letters

Regarding the photo:
  - Have an aspect ratio of 3:4
  - Have a resolution of at least 1125 px by 1500 px
  - Have adequate lighting; use flash if necessary

'''

interface = gr.Interface(
    fn=interface_function,
    title="HMC-Grad: Handwritten Multiple-Choice Test Grader",
    description=desc,
    allow_flagging="never",
    inputs=[gr.Textbox(label="Correction Key", placeholder="ABCDABCDABCD"),
            gr.File(file_count="multiple", file_types=[".jpg"], label="Upload Image(s)")],
    outputs=[gr.File(label="Correctness"),
             gr.File(label="Scores"),
             gr.File(label="Item Analysis"),
             gr.File(label="Score Analysis"),
             gr.File(label="Merged Data"),
             ],
    examples=[
        ["ABCDXABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDA",(["sample_1.jpg"])],
        ["ABCDXABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDA",(["sample_2.jpg"])],
        ["ABCDXABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDABCDA",(["sample_1.jpg", "sample_2.jpg"])],
    ]
)

# Launch the interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://65d4d822e6a4e8af8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
